In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#載入數據集，one_hot是把数据转化为只有0和1的形式
mnist = input_data.read_data_sets("MNIST_data", one_hot = True) 

#會將資料集分批次（一次100张）放入神经网络进行训练，
#每一個批次的大小
batch_size = 100 

#計算一共有多少批次，训练集数量mnist.train.num_examples 
# // 在python中表示取商
n_batch = mnist.train.num_examples // batch_size      


'''
# 参数概要(视频上原版)
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)  #平均值，该方法记录这个值并给其名字
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stdev', stddev)       #标准差
        tf.summary.scalar('max', tf.reduce_max(var))  #最大值
        tf.summary.scalar('min', tf.reduce_min(var))  #最小值
        tf.summary.scalar('histogram', var)     #直方图
'''


'''
#定义几个全局变量以及一些参数(修改版)
global mean_scaler
global stddev_scaler
global max_scaler
global min_scaler
global histogram_scaler
def variableSummaries(var):
    with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            mean_scaler = tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        stddev_scaler = tf.summary.scalar('stddev',stddev)#标准差
        max_scaler = tf.summary.scalar('max',tf.reduce_max(var))#最大值
        min_scaler = tf.summary.scalar('min',tf.reduce_min(var))#最小值
        histogram_scaler = tf.summary.histogram('histogram',var)#直方图
'''
#命名空间
with tf.name_scope('input'):
    #定义兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
    x = tf.placeholder(tf.float32, [None, 784], name='x-input') # 28 * 28 = 784，None值变为100
    y = tf.placeholder(tf.float32, [None, 10], name='y-input') #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量

with tf.name_scope('layer'):   
    #创建一個簡單的神經網路 (只有输入层和輸出層，输入层784个神经元，输出层總共10個神經元，即十个标签)
    with tf.name_scope('wights'):#命名空间下面的名门空间
        W = tf.Variable(tf.zeros([784, 10]), name='W')  #权值
        # 分析网络运行过程中权值的变化
        #variable_summaries(W)
        '''
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(W)
            tf.summary.scalar('mean', mean)  #平均值，该方法记录这个值并给其名字
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(W - mean)))
        tf.summary.scalar('stdev', stddev)       #标准差
        tf.summary.scalar('max', tf.reduce_max(W))  #最大值
        tf.summary.scalar('min', tf.reduce_min(W))  #最小值
        tf.summary.scalar('histogram', W)     #直方图
        '''
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(W)
            meanW_scaler = tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(W-mean)))
        stddevW_scaler = tf.summary.scalar('stddev',stddev)#标准差
        maxW_scaler = tf.summary.scalar('max',tf.reduce_max(W))#最大值
        minW_scaler = tf.summary.scalar('min',tf.reduce_min(W))#最小值
        histogramW_scaler = tf.summary.histogram('histogram',W)#直方图
        
        
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name='b') 
        #variable_summaries(b)
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(b)
            mean_scaler = tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(b-mean)))
        stddev_scaler = tf.summary.scalar('stddev',stddev)#标准差
        max_scaler = tf.summary.scalar('max',tf.reduce_max(b))#最大值
        min_scaler = tf.summary.scalar('min',tf.reduce_min(b))#最小值
        histogram_scaler = tf.summary.histogram('histogram',b)#直方图
        
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x, W) + b
        #分析'wx_plus_b和'预测值prediction'没多大意义，这里只分析权值W和偏置值b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)   #预测值，用到softmax


#二次代價函數 : loss = mean((y - prediction)^2)
#loss = tf.reduce_mean(tf.square(y - prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction)) #交叉熵
    # tf.summary.scalar('loss',loss)  
    loss_scaler = tf.summary.scalar('loss',loss)
    
    # 分析的数值比较多的时候才有必要调用variable_summaries（）函数
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量 operator
init = tf.global_variables_initializer()


with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 結果存在一個 boolean 的变量correct_prediction中
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))                                                
    with tf.name_scope('accuracy'):
        # 求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         
        accuracy_scaler = tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
#merged = tf.summary.merge_all()     #报错的原因所在！！！


#下面这些与网络结构没有关系，只有上面这些才会     
# 開始training
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph) #文件夹存放的是sess.graph图的结构
    for epoch in range(51): #总共疊代51次，因为主要是使用tensorboard来画神经网络结构的
        for batch in range(n_batch):#每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs图片数据, batch_ys图片标签
            # mnist.train.next_batch(batch_size)是获取下一个一百张图片               
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)               
           # summary,_ = sess.run([merged,train_step], feed_dict = {x: batch_xs, y: batch_ys}) #报错的原因所在之二！！！
            # 一边训练，一边统计权值和偏置值的loss值和accuracy值
            
            # writer.add_summary(summary, epoch)    #报错的原因所在之三！！！
            meanW_metall,stddevW_metall,maxW_metall,minW_metall,histogramW_metall,mean_metall,stddev_metall,max_metall,min_metall,histogram_metall,loss_metall,accuracy_metall,_ = sess.run([meanW_scaler,stddevW_scaler,maxW_scaler,minW_scaler,histogramW_scaler,mean_scaler,stddev_scaler,max_scaler,min_scaler,histogram_scaler,loss_scaler,accuracy_scaler,train_step], feed_dict={x:batch_xs,y:batch_ys})
        writer.add_summary(meanW_metall,epoch)
        writer.add_summary(stddevW_metall,epoch)
        writer.add_summary(maxW_metall,epoch)
        writer.add_summary(minW_metall,epoch)
        writer.add_summary(histogramW_metall,epoch)
        writer.add_summary(mean_metall,epoch)
        writer.add_summary(stddev_metall,epoch)
        writer.add_summary(max_metall,epoch)
        writer.add_summary(min_metall,epoch)
        writer.add_summary(histogram_metall,epoch)
        writer.add_summary(loss_metall,epoch)
        writer.add_summary(accuracy_metall,epoch)
         
    
            
        #每做完一次 outer loop 計算一次准确率
        acc = sess.run(accuracy, feed_dict = {x: mnist.test.images, y: mnist.test.labels})
                   
        # str(epoch)周期数
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter =0, Testing Accuracy =0.8248
Iter =1, Testing Accuracy =0.8938
Iter =2, Testing Accuracy =0.9018
Iter =3,